## Objectives
* Starting with a simple model, try creating a more advanced CNN model that can be easily tuned
* Create models with varying depth (number of convolutional layers) to observe performance vs time consumption
* Apply various hyperparameter tuning techniques to CNN models

In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
from torchvision.transforms import Normalize, ToTensor
import torch.nn as nn  # neural network
import torch.optim as optim  # optimization layer
import torch.nn.functional as F  # activation functions
import matplotlib.pyplot as plt
import argparse
import time
from collections import OrderedDict

In [2]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, data_dir, csv_file, transform=None):
        self.data_dir = data_dir
        self.data_df = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = {label: index for index, label in enumerate("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")}

        
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.data_df.iloc[idx, 0])
        image = Image.open(img_name)
        label_str = self.data_df.iloc[idx, 1]
        label = self.label_mapping[label_str]
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 62)  # 62 classes for EMNIST ByClass dataset

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        # print(x.shape)
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Initialize your CNN model
model = CNN()

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define transforms for the dataset
data_transform = transforms.Compose([
    transforms.Resize((32, 32)),    # Resize images to (32, 32)
    transforms.Grayscale(), # Convert images to grayscale
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(20),
    transforms.ToTensor()           # Convert images to PyTorch tensors
])

# Create a custom dataset and dataloader
dataset = CustomDataset(data_dir='data/',
                        csv_file='data/english.csv',
                        transform=data_transform)

# Split dataset into train and test partitions
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders for train and test partitions
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 100
device = torch.device("cpu")
model.to(device)
model_path = '/Users/smritikumari/Desktop/ML_Project/hand_dataset_model/model.pt'

model.load_state_dict(torch.load(model_path))

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        # print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    
    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_dataset)
    print(f"Training Loss: {epoch_loss:.4f}")

    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", unit="batch"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy:.4f}")
torch.save(model.to(torch.device('cpu')).state_dict(), '/Users/smritikumari/Desktop/ML_Project/hand_dataset_model/model.pt')

Epoch 1/100: 100%|██████████| 86/86 [00:31<00:00,  2.71batch/s]


Training Loss: 0.0564


Testing: 100%|██████████| 22/22 [00:07<00:00,  3.01batch/s]


Accuracy on test set: 0.8944


Epoch 2/100: 100%|██████████| 86/86 [00:31<00:00,  2.73batch/s]


...

Accuracy on test set: 0.9941


Epoch 100/100: 100%|██████████| 86/86 [00:31<00:00,  2.76batch/s]


Training Loss: 0.0000


Testing: 100%|██████████| 22/22 [00:07<00:00,  3.06batch/s]

Accuracy on test set: 0.9941


In [11]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os 

# def emnist_byclass_label_to_char(label):
#     if label < 10:
#         return str(label)  # Digits 0-9
#     elif label < 36:
#         return chr(label + 55)  # Capital letters A-Z (ASCII 65-90)
#     else:
#         return chr(label + 61)  # Small letters a-z (ASCII 97-122)

def emnist_byclass_label_to_char(label):
        # Decode label using the label mapping
    label_mapping = {label: index for index, label in enumerate("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")}
    return list(label_mapping.keys())[list(label_mapping.values()).index(label)]

def infer(net, test_folder):
    '''
    Returns test accuracy
    
        Parameters:
            net (CNN): a trained model
            args (ArgumentParser): hyperparameters
        
        Returns:
            test_acc (float): test accuracy of a trained model
    '''

    # net = CNN()
    
    # net.load_state_dict(torch.load(model_path))
    net.eval()

    def predict_image(image_path, model):
        image = Image.open(image_path)  # Convert to grayscale
        # Define transforms for the dataset
        transform = transforms.Compose([
            transforms.Resize((32, 32)),    # Resize images to (32, 32)
            transforms.Grayscale(),         # Convert images to grayscale
            transforms.ToTensor()           # Convert images to PyTorch tensors
        ])
        image = transform(image)
        image = torch.unsqueeze(image, 0)  # Add batch dimension
        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output, 1)
            return predicted.item()


    # Iterate through each image in the folder
    for filename in os.listdir(test_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(test_folder, filename)
            predicted_label = predict_image(image_path, net)
            predicted_char = emnist_byclass_label_to_char(predicted_label)
            print(f"you wrote: for {filename}: {predicted_char}")

In [12]:
#Test inference code
model_path = '/Users/smritikumari/Desktop/ML_Project/hand_dataset_model/model.pt'
test_folder = '/Users/smritikumari/Desktop/ML_Project/hand_dataset_model/test_imgs/'
infer(model, test_folder)

you wrote: for h.png: h
you wrote: for i.png: i
you wrote: for Z.png: Z
you wrote: for AA.png: 1
you wrote: for O.png: O
you wrote: for PP.png: Y
you wrote: for B.png: B
you wrote: for C.png: C
you wrote: for BB.png: R
you wrote: for 5.png: 5
you wrote: for A.png: A
you wrote: for w.png: w
you wrote: for 6.png: 6
you wrote: for D.png: D
you wrote: for 2.png: 2
you wrote: for s.png: E
you wrote: for e.png: Q
you wrote: for g.png: 5
you wrote: for P.png: F
you wrote: for CC.png: C
you wrote: for Q.png: Q
you wrote: for f.png: f


## Observation
As neural nets became more complex with more nodes:
   * execution time got extended significantly
   * surprisingly, train loss and validation loss did not change much with the adam optimizer
   * train loss and validation loss improved gradually with the SGD optimizer
   
Side Notes
   * adam optimizer had tendency to overfit data in all models
   * building a model is finding the right balance between performance and efficiency
   
   
## Conclusion
After certain number of convolutional layers, the difference in performance between models is quite insignificant; however, the execution time tends to always increase with the depth, causing unwanted inefficiency. Therefore, the complexity of a convolutional neural network does not justify how efficient it is. 

In my opinion, a combination of random searching and grid searching to find the optimal number of convolutional layers would be a good approach in reaching the ideal balance between performance and efficiency.